Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
link = r'https://en.wikipedia.org/wiki/Arrondissements_of_Paris'
districts = pd.read_html(link)[2]
districts.rename(columns={'Name': 'Neighborhood'}, inplace=True)
districts = districts[['Neighborhood']]

<a id='item1'></a>

In [3]:
print('The dataframe has {} districts '.format(districts.shape[0]))

The dataframe has 17 districts 


In [4]:
CLIENT_ID = 'KCQMV33TOWL4KANF4ZHTKQIB15DZHU5JM01EEMYDZCXL4OAO' # your Foursquare ID
CLIENT_SECRET = 'F2LC1T2HLQ3UVXE2EB43FLM2ZRY0HK5Q5E23OL3RKVN2VR3L' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KCQMV33TOWL4KANF4ZHTKQIB15DZHU5JM01EEMYDZCXL4OAO
CLIENT_SECRET:F2LC1T2HLQ3UVXE2EB43FLM2ZRY0HK5Q5E23OL3RKVN2VR3L


In [5]:
geolocator = Nominatim(user_agent="paris_locator")
location = geolocator.geocode('Paris, France')
latitude = location.latitude
longitude = location.longitude

In [6]:
def get_location_district(df):
    latitudes, longitudes = [], []
    geolocator = Nominatim(user_agent="paris_explorer")
    for district in df['Neighborhood']:
        #print(district)
        location = geolocator.geocode('{}, Paris, France'.format(' '.join(district.split(' ')[:2])))
        latitudes.append(location.latitude)
        longitudes.append(location.longitude)
    df['Latitude'] = latitudes
    df['Longitude'] = longitudes
    return df

In [7]:
districts = get_location_district(districts)
districts

,Neighborhood,Latitude,Longitude
0,"Louvre, Bourse, Temple, Hôtel-de-Ville",48.867250,2.343937
1,Panthéon,48.846191,2.346079
2,Luxembourg,48.850433,2.332951
3,Palais-Bourbon,48.861596,2.317909
4,Élysée,48.846644,2.369830
5,Opéra,48.870645,2.332330
6,Entrepôt,48.876106,2.359910
7,Popincourt,48.858416,2.379703
8,Reuilly,48.839615,2.395752
9,Gobelins,48.832397,2.355583


<a id='item2'></a>

## 2. Explore Neighborhoods in Manhattan

#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *manhattan_venues*.

In [9]:
LIMIT = 100
radius = 2000

paris_venues = getNearbyVenues(names=districts['Neighborhood'],
                                   latitudes=districts['Latitude'],
                                   longitudes=districts['Longitude']
                                  )



Louvre, Bourse, Temple, Hôtel-de-Ville
Panthéon
Luxembourg
Palais-Bourbon
Élysée
Opéra
Entrepôt
Popincourt
Reuilly
Gobelins
Observatoire
Vaugirard
Passy
Batignolles-Monceau
Butte-Montmartre
Buttes-Chaumont
Ménilmontant


#### Let's check the size of the resulting dataframe

In [10]:
print(paris_venues.shape)
paris_venues.head()

(1139, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Louvre, Bourse, Temple, Hôtel-de-Ville",48.86725,2.343937,Pizzeria Popolare,48.868074,2.343379,Pizza Place
1,"Louvre, Bourse, Temple, Hôtel-de-Ville",48.86725,2.343937,Le Nom M'échappe,48.868125,2.341673,Bistro
2,"Louvre, Bourse, Temple, Hôtel-de-Ville",48.86725,2.343937,Nose,48.866242,2.344885,Perfume Shop
3,"Louvre, Bourse, Temple, Hôtel-de-Ville",48.86725,2.343937,Filakia,48.865646,2.345619,Souvlaki Shop
4,"Louvre, Bourse, Temple, Hôtel-de-Ville",48.86725,2.343937,Librairie Gourmande,48.866951,2.343996,Bookstore


Let's check how many venues were returned for each neighborhood

In [11]:
paris_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Batignolles-Monceau,48,48,48,48,48,48
Butte-Montmartre,71,71,71,71,71,71
Buttes-Chaumont,38,38,38,38,38,38
Entrepôt,100,100,100,100,100,100
Gobelins,55,55,55,55,55,55
"Louvre, Bourse, Temple, Hôtel-de-Ville",100,100,100,100,100,100
Luxembourg,84,84,84,84,84,84
Ménilmontant,66,66,66,66,66,66
Observatoire,29,29,29,29,29,29


#### Let's find out how many unique categories can be curated from all the returned venues

In [12]:
print('There are {} uniques categories.'.format(len(paris_venues['Venue Category'].unique())))

There are 197 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [13]:
# one hot encoding
paris_onehot = pd.get_dummies(paris_venues[['Venue Category']], prefix="", prefix_sep="")

paris_onehot['Neighborhood'] = paris_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [paris_onehot.columns[-1]] + list(paris_onehot.columns[:-1])
paris_onehot = paris_onehot[fixed_columns]
paris_columns = paris_onehot.columns
"""for column in paris_columns:
    if 'restaurant' in column.lower():
        paris_onehot['Restaurant'] += paris_onehot[column]
        #paris_onehot.drop(column, inplace=True)"""

paris_onehot.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bar,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brasserie,Breakfast Spot,Breton Restaurant,Brewery,Bridge,Building,Burger Joint,Bus Stop,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Canal Lock,Candy Store,Ch'ti Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Dive Bar,Doner Restaurant,Donut Shop,Electronics Store,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Newsstand,Nightclub,Noodle House,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Resort,Restaurant,Rock Club,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Skating Rink,Soup Place,South American Restaurant,Southwestern French Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sports Bar,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,"Louvre, Bourse, Temple, Hôtel-de-Ville",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Louvre, Bourse, Temple, Hôtel-de-Ville",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Louvre, Bourse, Temple, Hôtel-de-Ville",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

And let's examine the new dataframe size.

In [14]:
paris_onehot.shape

(1139, 198)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [15]:
paris_grouped = paris_onehot.groupby('Neighborhood').mean().reset_index()
paris_grouped

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bar,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brasserie,Breakfast Spot,Breton Restaurant,Brewery,Bridge,Building,Burger Joint,Bus Stop,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Canal Lock,Candy Store,Ch'ti Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Dive Bar,Doner Restaurant,Donut Shop,Electronics Store,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Newsstand,Nightclub,Noodle House,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Resort,Restaurant,Rock Club,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Skating Rink,Soup Place,South American Restaurant,Southwestern French Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sports Bar,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Batignolles-Monceau,0.000000,0.000000,0.000000,0.000000,0.000000,0.020833,0.00,0.000000,0.000000,0.020833,0.000000,0.062500,0.020833,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020833,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.020833,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.020833,0.000000,0.020833,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.041667,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.166667,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.020833,0.000000,0.000000,0.000000,0.020833,0.000000,0.00,0.000000,0.020833,0.000000,0.020833,0.020833,0.00,0.00,0.00,0.083333,0.000000,0.00,0.000000,0.000000,0.020833,0.020833,0.041667,0.020833,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.020833,0.000000,0.020833,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000

#### Let's confirm the new size

In [16]:
paris_grouped.shape

(17, 198)

In [17]:
paris_cafe = paris_grouped[['Neighborhood', 'Café']]

<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [18]:
# set number of clusters
kclusters = 4

paris_grouped_clustering = paris_cafe.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(paris_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 3, 0, 2, 3, 0, 3, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [19]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

paris_merged = paris_cafe.copy()
# add clustering labels
paris_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
paris_merged = paris_merged.join(districts.set_index('Neighborhood'), on='Neighborhood')

paris_merged.head() # check the last columns!

,Neighborhood,Café,Cluster Labels,Latitude,Longitude
0,Batignolles-Monceau,0.020833,0,48.881452,2.316667
1,Butte-Montmartre,0.028169,0,48.890012,2.346467
2,Buttes-Chaumont,0.026316,0,48.878396,2.381201
3,Entrepôt,0.040000,3,48.876106,2.359910
4,Gobelins,0.018182,0,48.832397,2.355583


Finally, let's visualize the resulting clusters

In [20]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = ['red', 'blue', 'green', 'yellow', 'black']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(paris_merged['Latitude'], paris_merged['Longitude'], paris_merged['Neighborhood'], paris_merged['Cluster Labels']):
    if np.isnan(cluster):
        continue
    cluster = int(cluster)
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [21]:
paris_merged.loc[paris_merged['Cluster Labels'] == 0, paris_merged.columns[[1] + list(range(4, paris_merged.shape[1]))]]

,Café,Longitude
0,0.020833,2.316667
1,0.028169,2.346467
2,0.026316,2.381201
4,0.018182,2.355583
7,0.015152,2.383374
9,0.030000,2.332330
10,0.027778,2.317909


#### Cluster 2

In [22]:
paris_merged.loc[paris_merged['Cluster Labels'] == 1, paris_merged.columns[[1] + list(range(4, paris_merged.shape[1]))]]

,Café,Longitude
13,0.073171,2.379703
14,0.069767,2.395752


#### Cluster 3

In [23]:
paris_merged.loc[paris_merged['Cluster Labels'] == 2, paris_merged.columns[[1] + list(range(4, paris_merged.shape[1]))]]

,Café,Longitude
5,0.01,2.343937
12,0.00,2.280983
15,0.00,2.300383
16,0.00,2.369830


#### Cluster 4

In [24]:
paris_merged.loc[paris_merged['Cluster Labels'] == 3, paris_merged.columns[[1] + list(range(4, paris_merged.shape[1]))]]

,Café,Longitude
3,0.040000,2.359910
6,0.035714,2.332951
8,0.034483,2.323962
11,0.050000,2.346079


# Conclusion

As we can see, in order to avoid the competition at the first stage, we have to open cafés somewhere in cluster 3.